In [1]:
import torch
import torch.nn as nn
from torchvision.datasets import MNIST
from torchvision.transforms import Compose, ToTensor, Lambda
from torch.optim import Adam, SGD
from torch.utils.data import DataLoader
import numpy as np
import random
from skimage.filters import gaussian
from sklearn.preprocessing import OneHotEncoder

In [2]:
# در این بخش داده هارا نرمالیزه میکنیم . و فلت میکنیم و آنها را به تنسور تبدیل میکنیم. بچ هایی که تعریف کردیم به سایز کل دیتاست ما هستند و تمامی دادگان دیتاست را یکجا پردازش میکنیم. 
def Dataloader():
    def normalize_data(x):

        x_min = torch.min(x, dim=1, keepdim=True)[0]
        x_max = torch.max(x, dim=1, keepdim=True)[0]
        x_norm = torch.sub(x, x_min)
        x_norm = torch.div(x_norm, x_max - x_min)
        x_ = torch.flatten(x)
        return x_

    transform = Compose([ToTensor(),Lambda(normalize_data)])

    train_loader = DataLoader(MNIST('./data/', train=True,download=True,transform=transform),batch_size=50000, shuffle=True)
    test_loader = DataLoader(MNIST('./data/', train=False,download=True,transform=transform),batch_size=10000, shuffle=False)
    x, y = next(iter(train_loader))
    x, y = x.cuda(), y.cuda()
    x_test, y_test = next(iter(test_loader))
    x_test, y_test = x_test.cuda(), y_test.cuda()
    return x,y,x_test,y_test

In [4]:
# در اینجا تولید ماسک دادگان و تولید تصویر هیبرید را در یک تابع انجام میدهیم. 
import torchvision.transforms as transforms
transfrom_tensor = transforms.ToTensor()
def data_gen(x,batch_size):
    random_image = np.random.randint(2, size=(28,28)).astype(np.float32)
    random_image = gaussian(random_image, sigma=2.5)
    # ابتدا یک تصویر ردنوم به سایز تصویر ورودی تولید میکنیم و با انخاب یک آستانه ی مناسب، ماسک شامل صفر و یک از تصویر رندوم تولید شده را درست میکنیم. 
    mask = (random_image > 0.5).astype(np.float32)
    mask = torch.from_numpy(mask)
    flattened_mask = mask.view(-1)
    expanded_mask = flattened_mask.expand(x.size(0), -1).to('cuda')
    x = x.to('cuda')
    hybrid_img = x*expanded_mask + x[torch.randperm(x.size(0))] * (1-expanded_mask)
    return hybrid_img


In [14]:
class AFNet(torch.nn.Module):
# یک شبکه ی سه لایه مخفی تعریف میکنیم که سایز ورودی آن به ابعاد تصویر فلت شده است. 
    def __init__(self):
        super().__init__()
        self.layers = []
        self.layers.append (Layer(784, 600).cuda())
        self.layers.append (Layer(600, 400).cuda())
        self.layers.append (Layer(400, 300).cuda())
# با توجه به شبکه ای که تعریف کردیم و کلاس لایه ای که جداگانه تعریف کردیم، بخش آموزش شبکه، ورودی های منفی و مثبت را دریافت میکند و با استفاده از کلاس بعدی آنهارا آموزش میدهید.
    def train(self, positive_data, negative_data):
        out_positive, out_negative = positive_data, negative_data
        for layer in self.layers:
            out_positive, out_negative = layer.train(out_positive, out_negative)
# در اینجا بخش پیش بینی شبکه آورده شده است. اینجا خروجی هر لایه از شبکه در یک لیست ریخته میشود که خروجی هر لایه تبدیل ووردی آن به یک بردار نمایش دیگر از ورودی است که به طبقه بند ما داده می شود.
    def predict(self, x):
        out_layer = x
        out_layers = torch.Tensor([]).cuda()
        for layer in self.layers:
            out_layer = layer(out_layer)
            out_layers = torch.cat([out_layers,out_layer],1)
        return out_layers
# از آنجایی که در این روش ما با هر لایه و آموزش تابع هزینه ی محلی آن سر و کار داریم، یک کلاس لایه ی جداگانه تعریف میشود. که خروجی را به صورت لایه به لایه تولید میکند و به کلاس شبکه ی اصلی میدهد. 
class Layer(nn.Linear):
    def __init__(self, in_features, out_features,bias=True):
        super().__init__(in_features, out_features, bias)
        self.relu = torch.nn.ReLU()
        self.optimizer = Adam(self.parameters(), lr=0.05)

    def forward(self, x):
        x_norm = torch.nn.functional.normalize(x, p=2, dim=1, eps=1e-5)
        x_out = self.relu(torch.mm(x_norm, self.weight.T) +self.bias.unsqueeze(0))
        return x_out
# در بخش آموزش شبکه با توجه به معیار خوب بودنی که تعریف میشود، و آستانه ای که انتخاب میکنیم، شبکه سعی میکند فعالیت بخش دادگان مثبت را تا جای امکان بالا نگه دارد و در مورد دادگان منفی، برعکس عمل میکند. 
# در نهایت هم بعد از آموزش ایپاک های مورد نظر، ورودی از لایه ی مورد نظر رد میشود و به بخش آموزش شبکه ی اصلی داده میشود. 
    def train(self, x_positive, x_negative):
        num_epochs = 1000
        threshold = 2.0
        for i in range(num_epochs):
            loss = torch.log(1 + torch.exp(torch.cat([ threshold-(self.forward(x_positive).pow(2).mean(1)),(self.forward(x_negative).pow(2).mean(1))-threshold]))).mean()
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
        out_layer_pos = self.forward(x_positive)
        out_layer_neg = self.forward(x_negative)
        return out_layer_pos.detach(),out_layer_neg.detach()

torch.manual_seed(1234)
x_train,y_train,x_test,y_test = Dataloader()
x_negative_train = data_gen(x_train,50000)
net = AFNet()
net.train(x_train, x_negative_train)




In [55]:
# این تابع برای تبدیل برچسب های داده ی امنیست، به یک بردار وان هات است که در ادامه مرود استفاده قرار میگیرد و با برچسب های تولیدی توسط طبقه بند مقایسه میشود. 
def one_hot(y):
    y = y.to('cpu')
    y = y.long()
    one_hot_y = torch.nn.functional.one_hot(y, num_classes=10)
    return one_hot_y.to(torch.float32)

In [ ]:
class LinearClassifier(nn.Module):
    def __init__(self, input_dimension):
        super().__init__()
        self.softmax = nn.Softmax(dim=1)
        self.linear = torch.nn.Linear(input_dimension, 10).cuda()
        self.optimizer = Adam(self.parameters(), lr=0.05)
        self.criteria = nn.CrossEntropyLoss()
# در اینجا بخش پیشبینی طبقه بند، خروجی های به هم پوست شده ی شبکه ی قبلی را میگیرد و از یک لایه ی خطی عبور میدهد و به سافتمکس میدهد تا برچسب هر داده با بیشترین احتمال وقوع تخمین زده شود. 
    def predict(self,x):
        represent_vector = net.predict(x)
        y_h = self.forward(represent_vector)
        soft_out = self.softmax(y_h)
        return soft_out.argmax(1)

    def forward(self,x):
        return self.linear(x)
# در بخش آموزش شبکه هم از معیار آنتروپی استفاده شده است که برچسب های وان هات شده را با برچسب های تولید شده از ادغام شبکه ی قبلی با طبقه بند مقایسه میکند و سعی میکند این اختلاف را به خداقل برساند. 
    def train(self, x,y):
        num_epochs = 300
        for i in range (num_epochs):
            y_onehot = one_hot(y).cuda()
            represent_vector = net.predict(x)
            y_transformed = self.forward(represent_vector)
            loss = self.criteria(y_transformed,y_onehot)
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            if i % 30 == 0:
                print('loss', loss)
input_dimension = 300+400+600
torch.manual_seed(1234)
classifier = LinearClassifier(input_dimension)
classifier.train(x_train,y_train)




loss tensor(2.8166, device='cuda:0', grad_fn=<DivBackward1>)
loss tensor(2.3351, device='cuda:0', grad_fn=<DivBackward1>)
loss tensor(1.0440, device='cuda:0', grad_fn=<DivBackward1>)
loss tensor(0.5028, device='cuda:0', grad_fn=<DivBackward1>)
loss tensor(0.2872, device='cuda:0', grad_fn=<DivBackward1>)
loss tensor(0.1895, device='cuda:0', grad_fn=<DivBackward1>)


In [52]:
y_pred_train = classifier.predict(x_train)
y_true_train = y_train.to('cpu')
y_pred_train = y_pred_train.to('cpu')
train_accuracy = torch.sum(y_pred_train == y_true_train).item() / len(y_train)
print('Train Accuracy' , train_accuracy)

Train Accuracy 0.9589


In [53]:
y_test_t = y_test.to('cpu')
y_pred_te = classifier.predict(x_test)
y_pred_t = y_pred_te.to('cpu')
test_accuracy = torch.sum(y_pred_t == y_test_t).item() / len(y_test)
print('Test Accuracy',test_accuracy)

Test Accuracy 0.9372
